## Fine Tuning of VGG16 for Biofuel Material Cassification

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as kr

%matplotlib inline

In [2]:
DATASET_ROOT = './dataset/'
TRAIN_FOLDER = 'train'
VALIDATE_FOLDER = 'validate'
TEST_FOLDER = 'test'

In [3]:
#=========[IMAGE PREPERATION]
# perhaps check: https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/
IDG = kr.preprocessing.image.ImageDataGenerator
train_set = IDG().flow_from_directory(DATASET_ROOT + TRAIN_FOLDER, target_size=(244,244), classes=['biomass', 'non_biomass'], batch_size=15);
validate_set = IDG().flow_from_directory(DATASET_ROOT + VALIDATE_FOLDER, target_size=(244,244), classes=['biomass', 'non_biomass'], batch_size=6);
test_set = IDG().flow_from_directory(DATASET_ROOT + TEST_FOLDER, target_size=(244,244), classes=['biomass', 'non_biomass'], batch_size=15);

Found 2411 images belonging to 2 classes.
Found 917 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [6]:
vgg16 = kr.applications.vgg16
vgg16_model = vgg16.VGG16()

In [7]:
#========[BUILD STAGE] - We need our last layer to be a binary output, so we edit VGG16

# First create new sequential model based of VGG16
svgg16_model = kr.models.Sequential()
for layer in vgg16_model.layers:
    svgg16_model.add(layer)

# remove last layer
svgg16_model.layers.pop()

# disable trainable for each layer
for layer in vgg16_model.layers:
    layer.trainable = False
    
# add new binary layer
svgg16_model.add(kr.layers.Dense(2, activation='softmax'))

In [8]:
#========[TRAIN STAGE] -

svgg16_model.compile(kr.optimizers.Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])
svgg16_model.fit_generator(train_set, steps_per_epoch=4, validation_data=validate_set, validation_steps=4, epochs=5, verbose=2)

Epoch 1/5


ValueError: Error when checking input: expected input_2 to have shape (224, 224, 3) but got array with shape (244, 244, 3)

### Cells Below Are for Testing Purposes

In [ ]:
####### DONT RUN - THIS IS THE CELL FOR OUTPUT TESTING ########
train_set[0]

In [ ]:
####### DONT RUN - THIS IS FOR TESTING PURPOSES ########

# load the model
vgg16 = kr.applications.vgg16
model = vgg16.VGG16()
# load an image from file
image = kr.preprocessing.image.load_img('dataset/train/biomass/cardboard189.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = kr.preprocessing.image.img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = vgg16.preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = vgg16.decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))